In [1]:
# notebook settings
%load_ext autoreload
%autoreload 2

# external imports
import numpy as np
import sympy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
from copy import copy
from math import floor

# internal imports
from pympc.geometry.polyhedron import Polyhedron
from pympc.dynamics.discrete_time_systems import LinearSystem, AffineSystem, PieceWiseAffineSystem
from pympc.control.hybrid_benchmark.controllers import HybridModelPredictiveController
from pympc.plot import plot_input_sequence, plot_state_trajectory, plot_output_trajectory
from pympc.control.hybrid_benchmark.utils import get_constraint_set, remove_redundant_inequalities_fast, convex_hull_method_fast

# Problem set-up

In [2]:
# numeric parameters of the system
m = 1.
r = .1
I = .4*m*r**2.
d = .4
l = .3
mu = .2
g = 10.
h = .05

kn = 500.
bn = 100.
bt = 100.

In [3]:
# symbolic state
xb, yb, tb = sp.symbols('xb yb tb') # position of the ball
xf, yf = sp.symbols('xf yf') # position of the floor
xdb, ydb, tdb = sp.symbols('xdb ydb tdb') # velocity of the ball
xdf, ydf = sp.symbols('xdf ydf') # velocity of the floor
x = sp.Matrix([
    xb, yb, tb,
    xf, yf,
    xdb, ydb, tdb,
    xdf, ydf
])

# symbolic input
xd2f, yd2f = sp.symbols('xd2f yd2f') # acceleration of the floor
u = sp.Matrix([
    xd2f, yd2f
])

# contact forces
ftf, fnf = sp.symbols('ftf fnf') # floor force
ftc, fnc = sp.symbols('ftc fnc') # ceiling force

In [4]:
# ball dynamics
xd2b = ftf/m - ftc/m
yd2b = fnf/m - fnc/m - g
td2b = r*(ftf+ftc)/I

# state update
xd = sp.Matrix([
    xdb, ydb, tdb,
    xdf, ydf,
    xd2b, yd2b, td2b,
    xd2f, yd2f
])

In [5]:
# gap functions
penetration_floor = yf - yb + m*g/kn
penetration_ceiling = yb - d - m*g/kn + 2.*r

# relative tangential velocity
sliding_velocity_floor = xdb + r*tdb - xdf
sliding_velocity_ceiling = xdb - r*tdb

# force with the floor
fnf_penetration = kn*(penetration_floor) - bn*(ydb - ydf)
ftf_penetration = - bn*sliding_velocity_floor

# force with the ceiling
fnc_penetration = - kn*(penetration_ceiling) + bn*ydb
ftc_penetration = bn*sliding_velocity_ceiling

# ball distance to boundaries
ball_on_floor = sp.Matrix([
    xb - xf - l,
    xf - xb - l
])
ball_on_ceiling = sp.Matrix([
    xb - l,
    - xb - l
])

In [6]:
# state bounds
x_max = np.array([
    l, d + m*g/kn - 2.*r + d, 1.2*np.pi, # ball config
    l, l/2.,            # floor config
    10., 10., 20.,     # ball vel
    10., 10.          # floor vel
])
x_min = - x_max

# input bounds
u_max = np.array([
    30., 30.,          # floor acc
])
u_min = - u_max

# domain bounds
xu = x.col_join(u)
xu_min = np.concatenate((x_min, u_min))
xu_max = np.concatenate((x_max, u_max))

In [7]:
# discrete time dynamics in mode 1
# (ball in the air)

# set forces to zero
f_m1 = {ftf: 0., fnf: 0., ftc: 0., fnc: 0.}

# get dynamics
xd_m1 = xd.subs(f_m1)
S1 = AffineSystem.from_symbolic_continuous(x, u, xd_m1, h, 'zero_order_hold')

# build domain
D1 = Polyhedron.from_bounds(xu_min, xu_max)

# penetration <= 0 with floor and ceiling
D1.add_symbolic_inequality(xu, sp.Matrix([penetration_floor]))
D1.add_symbolic_inequality(xu, sp.Matrix([penetration_ceiling]))

# check domain
assert D1.bounded
assert not D1.empty

In [8]:
# discrete time dynamics in mode 2
# (ball in contact the floor, floor pulling)

# get dynamics
S2 = copy(S1)

# build domain
D2 = Polyhedron.from_bounds(xu_min, xu_max)

# penetration >= 0 with floor
D2.add_symbolic_inequality(xu, sp.Matrix([-penetration_floor]))

# penetration <= 0 with ceiling
D2.add_symbolic_inequality(xu, sp.Matrix([penetration_ceiling]))

# ball not falling down the floor
D2.add_symbolic_inequality(xu, ball_on_floor)

# floor pulling
D2.add_symbolic_inequality(xu, sp.Matrix([fnf_penetration]))

# check domain
assert D2.bounded
assert not D2.empty

In [9]:
# discrete time dynamics in mode 3
# (ball sticking with the floor, not in contact with the ceiling)

# enforce sticking
f_m3 = {ftf: ftf_penetration, fnf: fnf_penetration, ftc: 0., fnc: 0.}

# get dynamics
xd_m3 = xd.subs(f_m3)
S3 = AffineSystem.from_symbolic_continuous(x, u, xd_m3, h, 'zero_order_hold')

# build domain
D3 = Polyhedron.from_bounds(xu_min, xu_max)

# penetration >= 0 with floor
D3.add_symbolic_inequality(xu, sp.Matrix([-penetration_floor]))

# penetration <= 0 with ceiling
D3.add_symbolic_inequality(xu, sp.Matrix([penetration_ceiling]))

# ball not falling down the floor
D3.add_symbolic_inequality(xu, ball_on_floor)

# friction cone
D3.add_symbolic_inequality(xu, sp.Matrix([ftf_penetration - mu*fnf_penetration]))
D3.add_symbolic_inequality(xu, sp.Matrix([- ftf_penetration - mu*fnf_penetration]))

# floor pushing
D3.add_symbolic_inequality(xu, sp.Matrix([- fnf_penetration]))

# check domain
assert D3.bounded
assert not D3.empty

In [10]:
# discrete time dynamics in mode 4
# (ball sliding right on the floor, not in contact with the ceiling)

# enforce sliding
f_m4 = {ftf: -mu*fnf_penetration, fnf: fnf_penetration, ftc: 0., fnc: 0.}

# get dynamics
xd_m4 = xd.subs(f_m4)
S4 = AffineSystem.from_symbolic_continuous(x, u, xd_m4, h, 'zero_order_hold')

# build domain
D4 = Polyhedron.from_bounds(xu_min, xu_max)

# penetration >= 0 with floor
D4.add_symbolic_inequality(xu, sp.Matrix([-penetration_floor]))

# penetration <= 0 with ceiling
D4.add_symbolic_inequality(xu, sp.Matrix([penetration_ceiling]))

# ball not falling down the floor
D4.add_symbolic_inequality(xu, ball_on_floor)

# positive relative velocity
D4.add_symbolic_inequality(xu, sp.Matrix([ftf_penetration + mu*fnf_penetration]))
# D4.add_symbolic_inequality(xu, sp.Matrix([mu*fnf_penetration - bt*sliding_velocity_floor]))

# floor pushing
D4.add_symbolic_inequality(xu, sp.Matrix([- fnf_penetration]))

# check domain
assert D4.bounded
assert not D4.empty

In [11]:
# discrete time dynamics in mode 5
# (ball sliding left on the floor, not in contact with the ceiling)

# enforce sliding
f_m5 = {ftf: mu*fnf_penetration, fnf: fnf_penetration, ftc: 0., fnc: 0.}

# get dynamics
xd_m5 = xd.subs(f_m5)
S5 = AffineSystem.from_symbolic_continuous(x, u, xd_m5, h, 'zero_order_hold')

# build domain
D5 = Polyhedron.from_bounds(xu_min, xu_max)

# penetration >= 0 with floor
D5.add_symbolic_inequality(xu, sp.Matrix([-penetration_floor]))

# penetration <= 0 with ceiling
D5.add_symbolic_inequality(xu, sp.Matrix([penetration_ceiling]))

# ball not falling down the floor
D5.add_symbolic_inequality(xu, ball_on_floor)

# negative relative velocity
D5.add_symbolic_inequality(xu, sp.Matrix([mu*fnf_penetration - ftf_penetration]))
# D5.add_symbolic_inequality(xu, sp.Matrix([mu*fnf_penetration + bt*sliding_velocity_floor]))

# floor pushing
D5.add_symbolic_inequality(xu, sp.Matrix([- fnf_penetration]))

# check domain
assert D5.bounded
assert not D5.empty

In [12]:
# discrete time dynamics in mode 6
# (ball in contact the ceiling, ceiling pulling)

# get dynamics
S6 = copy(S1)

# build domain
D6 = Polyhedron.from_bounds(xu_min, xu_max)

# penetration <= 0 with floor
D6.add_symbolic_inequality(xu, sp.Matrix([penetration_floor]))

# penetration >= 0 with ceiling
D6.add_symbolic_inequality(xu, sp.Matrix([-penetration_ceiling]))

# ball not falling down the celing
D6.add_symbolic_inequality(xu, ball_on_ceiling)

# ceiling pulling
D6.add_symbolic_inequality(xu, sp.Matrix([fnc_penetration]))

# check domain
assert D6.bounded
assert not D6.empty

In [13]:
# discrete time dynamics in mode 7
# (ball sticking on the ceiling, not in contact with the floor)

# enforce sticking
f_m7 = {ftc: ftc_penetration, fnc: fnc_penetration, ftf: 0., fnf: 0.}

# get dynamics
xd_m7 = xd.subs(f_m7)
S7 = AffineSystem.from_symbolic_continuous(x, u, xd_m7, h, 'zero_order_hold')

# build domain
D7 = Polyhedron.from_bounds(xu_min, xu_max)

# penetration <= 0 with floor
D7.add_symbolic_inequality(xu, sp.Matrix([penetration_floor]))

# penetration >= 0 with ceiling
D7.add_symbolic_inequality(xu, sp.Matrix([-penetration_ceiling]))

# ball in contact with the ceiling
D7.add_symbolic_inequality(xu, ball_on_ceiling)

# friction cone
D7.add_symbolic_inequality(xu, sp.Matrix([ftc_penetration - mu*fnc_penetration]))
D7.add_symbolic_inequality(xu, sp.Matrix([- ftc_penetration - mu*fnc_penetration]))

# ceiling pushing
D7.add_symbolic_inequality(xu, sp.Matrix([- fnc_penetration]))

# check domain
assert D7.bounded
assert not D7.empty

In [14]:
# discrete time dynamics in mode 8
# (ball sliding right on the ceiling, not in contact with the floor)

# enforce sliding
f_m8 = {ftc: mu*fnc_penetration, fnc: fnc_penetration, ftf: 0., fnf: 0.}

# get dynamics
xd_m8 = xd.subs(f_m8)
S8 = AffineSystem.from_symbolic_continuous(x, u, xd_m8, h, 'zero_order_hold')

# build domain
D8 = Polyhedron.from_bounds(xu_min, xu_max)

# penetration <= 0 with floor
D8.add_symbolic_inequality(xu, sp.Matrix([penetration_floor]))

# penetration >= 0 with ceiling
D8.add_symbolic_inequality(xu, sp.Matrix([-penetration_ceiling]))

# ball in contact with the ceiling
D8.add_symbolic_inequality(xu, ball_on_ceiling)

# positive relative velocity
D8.add_symbolic_inequality(xu, sp.Matrix([mu*fnc_penetration - ftc_penetration]))
# D8.add_symbolic_inequality(xu, sp.Matrix([mu*fnc_penetration - bt*sliding_velocity_ceiling]))

# ceiling pushing
D8.add_symbolic_inequality(xu, sp.Matrix([- fnc_penetration]))

# check domain
assert D8.bounded
assert not D8.empty

In [15]:
# discrete time dynamics in mode 9
# (ball sliding left on the ceiling, not in contact with the floor)

# enforce sliding
f_m9 = {ftc: -mu*fnc_penetration, fnc: fnc_penetration, ftf: 0., fnf: 0.}

# get dynamics
xd_m9 = xd.subs(f_m9)
S9 = AffineSystem.from_symbolic_continuous(x, u, xd_m9, h, 'zero_order_hold')

# build domain
D9 = Polyhedron.from_bounds(xu_min, xu_max)

# penetration <= 0 with floor
D9.add_symbolic_inequality(xu, sp.Matrix([penetration_floor]))

# penetration >= 0 with ceiling
D9.add_symbolic_inequality(xu, sp.Matrix([-penetration_ceiling]))

# ball in contact with the ceiling
D9.add_symbolic_inequality(xu, ball_on_ceiling)

# negative relative velocity
D9.add_symbolic_inequality(xu, sp.Matrix([ftc_penetration + mu*fnc_penetration]))
# D9.add_symbolic_inequality(xu, sp.Matrix([mu*fnc_penetration + bt*sliding_velocity_ceiling]))

# ceiling pushing
D9.add_symbolic_inequality(xu, sp.Matrix([- fnc_penetration]))

# check domain
assert D9.bounded
assert not D9.empty

In [16]:
# list of dynamics
S_list = [S1, S2, S3, S4, S5, S6, S7, S8, S9]

# list of domains
D_list = [D1, D2, D3, D4, D5, D6, D7, D8, D9]

# PWA system
S = PieceWiseAffineSystem(S_list, D_list)

In [17]:
# controller parameters
N = 20
Q = np.diag([
    1., .0, .0,
    1., 1.,
    1., 1., .01,
    1., 1.
])*h
R = np.diag([
    .01, .001
])*h
P = np.zeros((S.nx, S.nx))

# terminal set and cost
X_N = Polyhedron.from_bounds(*[np.zeros(S.nx)]*2)
# xN_max = np.array([l]+[0.]*(S.nx-1))
# xN_min = xN_max
# X_N = Polyhedron.from_bounds(x_min, x_max)

In [18]:
controller = HybridModelPredictiveController(S, N, Q, R, P, X_N, method='Convex hull')
# controller.add_reachability_constraints(5)
# controller.prog.setParam('Heuristics', 0)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


In [19]:
# initial condition
x0 = np.array([
    0., 0., np.pi,
    0., 0.,
    0., 0., 0.,
    0., 0.
])

In [20]:
# controller.prog.setParam('MIPFocus', 2)
u_opt, x_opt, ms_opt, cost_opt = controller.feedforward(x0)

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 9560 rows, 4390 columns and 32180 nonzeros
Model has 200 quadratic objective terms
Variable types: 4210 continuous, 180 integer (180 binary)
Coefficient statistics:
  Matrix range     [2e-04, 5e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-05, 5e-02]
  Bounds range     [3e+00, 3e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 4938 rows and 2190 columns
Presolve time: 0.11s
Presolved: 4622 rows, 2200 columns, 17096 nonzeros
Presolved model has 192 quadratic objective terms
Variable types: 2042 continuous, 158 integer (158 binary)

Root relaxation: objective 4.156479e-03, 22870 iterations, 2.10 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00416    0   80          -    0.00416      -     -    2s
     0     0    0.00426    0   82          -    

 10649  6095 infeasible   66               -    0.02199      -  1356  610s
 10939  6225 infeasible   71               -    0.02226      -  1365  629s
 11244  6433    0.23361   51   41          -    0.02226      -  1380  650s
 11472  6541  postponed   57               -    0.02270      -  1421  670s
 11677  6646    0.40350   36   37          -    0.02298      -  1452  679s

Cutting planes:
  GUB cover: 9

Explored 11797 nodes (17218287 simplex iterations) in 679.19 seconds
Thread count was 4 (of 4 available processors)

Solution count 0

Solve interrupted
Best objective -, best bound 2.297602823527e-02, gap -


from pympc.control.hybrid_benchmark.branch_and_bound import branch_and_bound, best_first, depth_first
def solver(identifier, objective_cutoff):
    return controller.solve_relaxation(x0, identifier, objective_cutoff)
lb = {}
ub = {}
sol = {}
methods = [
    'Convex hull, lifted constraints',
    'Convex hull',
    'Big-M',
    'Traditional formulation'
]
for method in methods:
    print method
    controller = HybridModelPredictiveController(S, N, Q, R, P, X_N, method)
    sol_i, lb_i, ub_i = branch_and_bound(solver, depth_first, controller.explore_in_chronological_order)
    lb[method] = lb_i
    ub[method] = ub_i
    sol[method] = sol_i

In [ ]:
for i in range(S.nx):
    print 'x_'+str(i), sum(xt[i]*Q[i,i]*xt[i] for xt in x_opt)

for i in range(S.nu):
    print 'u_'+str(i), sum(ut[i]*R[i,i]*ut[i] for ut in u_opt)

In [ ]:
for i in range(S.nx):
    print 'x max_'+str(i), x_max[i], max(xt[i] for xt in x_opt)
    print 'x min_'+str(i), x_min[i], min(xt[i] for xt in x_opt)

for i in range(S.nu):
    print 'u max_'+str(i), u_max[i], max(ut[i] for ut in u_opt)
    print 'u min_'+str(i), u_min[i], min(ut[i] for ut in u_opt)

In [ ]:
plot_input_sequence(u_opt, h, (u_min, u_max))

In [ ]:
plot_state_trajectory(x_opt, h, (x_min, x_max))

for i in range(S.nx):
    print i, "min", min(xt[i] for xt in x)
    print i, "max", max(xt[i] for xt in x)
print x_min
print x_max

for i in range(S.nx):
    print sum(xt[i]*Q[i,i]*xt[i] for xt in x)
for i in range(S.nu):
    print sum(ut[i]*R[i,i]*ut[i] for ut in u)

In [ ]:
x0 = np.array([
    0., 0.2, 0.,
    0., 0.,
    0., 5., 5.,
    0., 0.
])
u_sim = [np.zeros(S.nu)]*100
x_sim, ms_sim = S.simulate(x0, u_sim)
print ms_sim

# Animation

In [ ]:
import meshcat
from meshcat.geometry import Box, Sphere, Cylinder, MeshLambertMaterial
from meshcat.animation import Animation
import meshcat.transformations as tf

In [ ]:
vis = meshcat.Visualizer()
# vis.jupyter_cell()
vis.open()

In [ ]:
tickness = .05
depth = .3
red = 0xff2222
blue = 0x2222ff
green = 0x22ff22
vis['ball'].set_object(
    Sphere(r),
    MeshLambertMaterial(color=blue)
)
vis['floor'].set_object(
    Box([depth, l*2., tickness]),
    MeshLambertMaterial(color=red)
)
vis['ceiling'].set_object(
    Box([depth, l*2., tickness]),
    MeshLambertMaterial(color=red)
)
vis['ball_orientation'].set_object(
    Cylinder(r/10., r),
    MeshLambertMaterial(color=green)
)

In [ ]:
anim = Animation()
for t, xt in enumerate(x_opt):
    with anim.at_frame(vis, t*h*300) as frame:
        frame['ball'].set_transform(
            tf.translation_matrix([0, xt[0], xt[1]+r-m*g/kn])
        )
        frame['floor'].set_transform(
            tf.translation_matrix([0, xt[3], xt[4]-tickness/2.])
        )
        frame['ceiling'].set_transform(
            tf.translation_matrix([0, 0, d+tickness/2.])
        )
        frame['ball_orientation'].set_transform(
            tf.translation_matrix([0, xt[0], xt[1]+r-m*g/kn]).dot(
                tf.rotation_matrix(xt[2], [1.,0.,0.])
            )
        )
vis.set_animation(anim)